In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from nltk import FreqDist
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
import matplotlib.pyplot as plt
sns.set_palette("pastel")
import ast
import networkx as nx
from bs4 import BeautifulSoup
import regex as re
import tqdm
import glob
import os
from fuzzywuzzy import fuzz

import multiprocessing
from multiprocessing import Process, Manager

import numexpr as ne

In [ ]:
citations_data = r"C:\Users\wb570559\Dropbox\AIR POLLUTION PROJECT\ANALYSIS\DATA\processed_data\citations_data_full\\"

In [ ]:
df_main = pd.read_csv(citations_data+r"citations_data_full_kanoon.csv").drop(["Unnamed: 0"], axis=1)

In [ ]:
df_main.head()

In [ ]:
len(df_main)

In [ ]:
len(df_main[df_main["acts_cited"]!="[]"])

In [ ]:
df_non_null = df_main[df_main["acts_cited"]!="[]"]

In [ ]:
preethikas_list = ["Water (Prevention and Control of Pollution) Act 1974 (Water Act)",
                  "Air (Prevention and Control of Pollution) Act 1981 (Air Act)",
                  "Environment (Protection) Act 1986 (EP Act)",
                  "E-Waste (Management) Rules 2016",
                  "Batteries (Management & Handling) Rules 2001",
                  "Battery Waste Management Rules 2020",
                  "Bio-Medical Waste Management Rules 2016",
                  "Plastic Waste Management Rules 2016",
                  "Solid Waste Management Rules 2016",
                  "Construction and Demolition Waste Management Rules 2016",
                  "Hazardous and Other Waste (Management and Transboundary Movement) Rules 2016",
                  "Manufacture, Storage and Import of Hazardous Chemicals Rules 1989 (MSIHC Rules)",
                  "Coastal Regulation Zone Notification 2019 (and related 2021 procedure for violation of the CRZ Notification)",
                  "Environment Impact Assessment Notification 2006",
                  "Wild Life (Protection) Act 1972",
                  "Forest (Conservation) Act 1980",
                  "Public Liability Insurance Act 1991",
                  "Biological Diversity Act 2002",
                  "National Green Tribunal Act 2010",
                  "Section 91 of the Civil Procedure Code",
                    "The Water (Prevention and Control of Pollution) Cess Act, 1977",
                    "The Forest (Conservation) Act, 1980",
                    "The Air (Prevention and Control of Pollution) Act, 1981",
                    "The Environment (Protection) Act, 1986",
                    "The Public Liability Insurance Act, 1991",
                    "The Biological Diversity Act, 2002"
                    ]

In [ ]:
more_acts_from_text = ['the air  prevention and control of pollution  act  1981',
 'article 226 in the constitution of india   1949',
 'the environment  protection  act  1986',
 'the water  prevention and control of pollution  act  1974',
 'section 21 in the air  prevention and control of pollution  act  1981',
 'section 25 in the water  prevention and control of pollution  act  1974',
 'article 21 in the constitution of india   1949',
 'section 31 in the air  prevention and control of pollution  act  1981',
 'section 31a in the air  prevention and control of pollution  act  1981',
 'section 44 in the water  prevention and control of pollution  act  1974',
 'section 37 in the air  prevention and control of pollution  act  1981',
 'section 33a in the water  prevention and control of pollution  act  1974',
 'section 15 in the air  prevention and control of pollution  act  1981',
 'section 26 in the water  prevention and control of pollution  act  1974',
 'section 14 in the air  prevention and control of pollution  act  1981',
 'section 29 in the air  prevention and control of pollution  act  1981',
 'section 3 in the air  prevention and control of pollution  act  1981',
 'section 33 in the water  prevention and control of pollution  act  1974',
 'the national green tribunal act  2010',
 'section 25 in the air  prevention and control of pollution  act  1981',
 'section 5 in the environment  protection  act  1986',
 'section 47 in the water  prevention and control of pollution  act  1974',
 'section 43 in the water  prevention and control of pollution  act  1974',
 'section 28 in the water  prevention and control of pollution  act  1974',
 'section 3 in the environment  protection  act  1986',
 'section 6 in the air  prevention and control of pollution  act  1981',
 'article 32 in the constitution of india   1949',
 'section 15 in the environment  protection  act  1986',
 'm c  mehta vs union of india   ors on 18 march  2004',
 'section 24 in the water  prevention and control of pollution  act  1974']

In [ ]:
final_list = preethikas_list+more_acts_from_text

In [ ]:
def Punctuation(string):

    # punctuation marks
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

    # traverse the given string and if any punctuation
    # marks occur replace it with null
    string = string.replace("\n", "")
    for x in string.lower():
        if x in punctuations:
            string = string.replace(x, " ")
            
    string = string.strip()
    string = string.replace("  ", " ")
    
    return string

In [ ]:
final_list = [Punctuation(ele.lower()) for ele in final_list]

In [ ]:
final_list

In [ ]:
final_list = ['water prevention and control of pollution act 1974 water act',
 'air prevention and control of pollution act 1981 air act',
 'environment protection act 1986 ep act',
 'e waste management rules 2016',
 'batteries management  handling rules 2001',
 'battery waste management rules 2020',
 'bio medical waste management rules 2016',
 'plastic waste management rules 2016',
 'solid waste management rules 2016',
 'construction and demolition waste management rules 2016',
 'hazardous and other waste management and transboundary movement rules 2016',
 'manufacture storage and import of hazardous chemicals rules 1989 msihc rules',
 'coastal regulation zone notification 2019 and related 2021 procedure for violation of the crz notification',
 'environment impact assessment notification 2006',
 'wild life protection act 1972',
 'forest conservation act 1980',

 'biological diversity act 2002',
 'national green tribunal act 2010',
 'section 91 of the civil procedure code',
 'the water prevention and control of pollution cess act 1977',
 'the forest conservation act 1980',
 'the air prevention and control of pollution act 1981',
 'the environment protection act 1986',
 'the public liability insurance act 1991',
 'the biological diversity act 2002',
 'the air prevention and control of pollution act 1981',

 'the environment protection act 1986',
 'the water prevention and control of pollution act 1974',
 'section 21 in the air prevention and control of pollution act 1981',
 'section 25 in the water prevention and control of pollution act 1974',

 'section 31 in the air prevention and control of pollution act 1981',
 'section 31a in the air prevention and control of pollution act 1981',
 'section 44 in the water prevention and control of pollution act 1974',
 'section 37 in the air prevention and control of pollution act 1981',
 'section 33a in the water prevention and control of pollution act 1974',
 'section 15 in the air prevention and control of pollution act 1981',
 'section 26 in the water prevention and control of pollution act 1974',
 'section 14 in the air prevention and control of pollution act 1981',
 'section 29 in the air prevention and control of pollution act 1981',
 'section 3 in the air prevention and control of pollution act 1981',
 'section 33 in the water prevention and control of pollution act 1974',
 'the national green tribunal act 2010',
 'section 25 in the air prevention and control of pollution act 1981',
 'section 5 in the environment protection act 1986',
 'section 47 in the water prevention and control of pollution act 1974',
 'section 43 in the water prevention and control of pollution act 1974',
 'section 28 in the water prevention and control of pollution act 1974',
 'section 3 in the environment protection act 1986',
 'section 6 in the air prevention and control of pollution act 1981',

 'section 15 in the environment protection act 1986',
 'm c mehta vs union of india  ors on 18 march 2004',
 'section 24 in the water prevention and control of pollution act 1974']

In [ ]:
df_non_null["acts_cited"] = df_non_null["acts_cited"].apply(lambda x: ast.literal_eval(x))

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
df_non_null["acts_cited"] = df_non_null["acts_cited"].progress_apply(lambda x: [Punctuation(ele.lower()) for ele in x])

In [ ]:
df_non_null["acts_cited"]

In [ ]:
def acts_matcher(list_green_acts, list_acts):
    
    #cache = 0
    list_matched_acts = []
    
    for act in list_acts:
        
        cache = 0
        flag = 0
        
        for green_act in list_green_acts:
            
            score = fuzz.token_set_ratio(act, green_act)
            
            if (score > 90 and score > cache):
                flag = 1
                cache = score
                matched_act = green_act
                
        if flag==1:
            
            list_matched_acts.append(matched_act)
            
        else:
            continue
            
    return list_matched_acts
                

In [ ]:
df_non_null["matched_acts"] = df_non_null["acts_cited"].progress_apply(lambda x: acts_matcher(final_list, x))

In [ ]:
df_non_null["matched_acts"].head()

In [ ]:
df_non_null.to_csv(citations_data+"matched_acts.csv")

In [ ]:
df_non_null["matched_acts"] = df_non_null["matched_acts"].apply(lambda x : str(x))

In [ ]:
df_non_null_matched = df_non_null[df_non_null["matched_acts"]!="[]"]

In [ ]:
len(df_non_null_matched)

In [ ]:
df_non_null_matched["1981_dum"] = df_non_null_matched["matched_acts"].apply(lambda x: 1 if "1981" in x else 0)

In [ ]:
df_non_null_matched["1981_dum"].describe()

In [ ]:
df_non_null_matched.to_csv(citations_data+"possible_air_corpus.csv")

In [20]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from nltk import FreqDist
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
import matplotlib.pyplot as plt
sns.set_palette("pastel")
import ast
import networkx as nx
from bs4 import BeautifulSoup
import regex as re
import tqdm
import glob
import os
from fuzzywuzzy import fuzz

import multiprocessing
from multiprocessing import Process, Manager

import numexpr as ne

In [21]:
citations_data = r"C:\Users\wb570559\Dropbox\AIR POLLUTION PROJECT\ANALYSIS\DATA\processed_data\citations_data_full\\"

In [22]:
df_non_null_matched = pd.read_csv(citations_data+"possible_air_corpus.csv").drop(["Unnamed: 0"], axis=1)

In [23]:
kanoon_metadata_path = r"C:\Users\wb570559\Dropbox\AIR POLLUTION PROJECT\INPUT\DATA\Law\cleaned_csv\cleaned_csv\\"

In [24]:
df_citations = pd.read_csv(kanoon_metadata_path+"df_citations.csv").drop(["Unnamed: 0"], axis=1)

In [25]:
df_cited_doc = pd.read_csv(kanoon_metadata_path+"df_cited_doc.csv").drop(["Unnamed: 0"], axis=1)

In [26]:
df_judges = pd.read_csv(kanoon_metadata_path+"df_judges.csv").drop(["Unnamed: 0"], axis=1)

In [27]:
df_meta =  pd.read_csv(kanoon_metadata_path+"df_meta.csv").drop(["Unnamed: 0"], axis=1)

C:\WBG\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
df_statutes = pd.read_csv(kanoon_metadata_path+"df_statutes.csv").drop(["Unnamed: 0"], axis=1)

In [29]:
df_citations.head()

,kanoon_id,eq_cite
0,1014770,[' (1900) ILR 22 All 367']
1,1031698,[' (1900) ILR 22 All 321']
2,103657,[' (1900) ILR 22 All 405']
3,1180938,[' (1900) ILR 22 All 340']
4,1215488,[' (1900) ILR 22 All 284']


In [30]:
df_cited_doc.head()

,kanoon_id,kanoon_ids_cited
0,1014770,[]
1,1031698,[]
2,103657,[707073]
3,1180938,[445276]
4,1215488,"[151798, 1705657, 1505714, 1626845]"


In [31]:
df_judges.head()

,kanoon_id,judge_court
0,1014770,"[(' Banerji', 'Allahabad High Court'), (' Aikm..."
1,1031698,"[(' Banerji', 'Allahabad High Court'), (' Aikm..."
2,103657,"[(' Knox', 'Allahabad High Court'), (' Blair',..."
3,1180938,"[(' Knox', 'Allahabad High Court'), (' Blair',..."
4,1215488,"[(' A Strachey', 'Allahabad High Court'), (' K..."


In [32]:
df_meta.head()

,kanoon_id,court,author,title,petitioner,respondent,doc_date,pre,judgment_or_order
0,1014770,Allahabad High Court,Banerji,"Jafri Begam vs Saira Bibi on 1 June, 1900",Jafri Begam,Saira Bibi,1900-06-01,NaN,1
1,1031698,Allahabad High Court,B A Aikman,Sheo Sampat Pande And Anr. vs Bandhu Prasad Mi...,Sheo Sampat Pande And Anr.,Bandhu Prasad Misr And Ors.,1900-04-17,NaN,1
2,103657,Allahabad High Court,NaN,Hafiz Abdul Rahim Khan vs Raja Hari Raj Singh ...,Hafiz Abdul Rahim Khan,Raja Hari Raj Singh,1900-06-14,NaN,1
3,1180938,Allahabad High Court,NaN,"Queen-Empress vs Narain Singh on 14 May, 1900",Queen-Empress,Narain Singh,1900-05-14,NaN,1
4,1215488,Allahabad High Court,Banerji,Bisheshur Dial And Anr. vs Ram Sarup on 3 Apri...,Bisheshur Dial And Anr.,Ram Sarup,1900-04-03,NaN,1


In [33]:
df_statutes.head()

,kanoon_id,doc_id
0,1014770,[]
1,1031698,[]
2,103657,[707073]
3,1180938,[445276]
4,1215488,"[151798, 1705657, 1505714, 1626845]"


In [34]:
df_non_null_matched.head()

,kanoon_id,acts_cited,matched_acts,1981_dum
0,1026316,"['the forest conservation act 1980', 'the indi...","['forest conservation act 1980', 'forest conse...",0
1,10391123,"['the environment protection act 1986', 'the c...",['the environment protection act 1986'],0
2,105012226,['section 9 in the wild life protection act 19...,"['wild life protection act 1972', 'wild life p...",0
3,1054793,['section 2 in the forest conservation act 198...,"['forest conservation act 1980', 'forest conse...",0
4,107410576,['section 91 in the code of criminal procedure...,['section 91 of the civil procedure code'],0


In [35]:
df_non_null_matched = pd.merge(df_non_null_matched, df_citations, on = "kanoon_id", how = "left")

In [36]:
df_non_null_matched = pd.merge(df_non_null_matched, df_cited_doc, on = "kanoon_id", how = "left")

In [37]:
df_non_null_matched = pd.merge(df_non_null_matched, df_judges, on = "kanoon_id", how = "left")

In [38]:
df_non_null_matched = pd.merge(df_non_null_matched, df_meta, on = "kanoon_id", how = "left")

In [39]:
df_non_null_matched.head()

,kanoon_id,acts_cited,matched_acts,1981_dum,eq_cite,kanoon_ids_cited,judge_court,court,author,title,petitioner,respondent,doc_date,pre,judgment_or_order
0,1026316,"['the forest conservation act 1980', 'the indi...","['forest conservation act 1980', 'forest conse...",0,[],"[871328, 867010, 72095, 824830, 1781078, 16655...","[(' Y.K.Sabharwal', 'Supreme Court of India'),...",Supreme Court of India,Y Sabharwal,T.N. Godavarman Thirumulpad vs Union Of India ...,T.N. Godavarman Thirumulpad,Union Of India & Ors,2005-09-26,CASE NO.:\nWrit Petition (civil) 2...,1
1,10391123,"['the environment protection act 1986', 'the c...",['the environment protection act 1986'],0,[],"[1353758, 182701402]",[],Kerala High Court,NaN,Kent Construction Pvt. Ltd vs Corporation Of K...,Kent Construction Pvt. Ltd,Corporation Of Kochi,2008-02-14,\n\n \nIN THE HIGH COURT OF KERALA AT ...,1
2,105012226,['section 9 in the wild life protection act 19...,"['wild life protection act 1972', 'wild life p...",0,[],"[672187, 1280344, 912273, 1290514, 702677, 178...",[],Madhya Pradesh High Court,NaN,Rekh Singh vs The State Of Madhya Pradesh on 2...,Rekh Singh,The State Of Madhya Pradesh,2019-01-02,...,0
3,1054793,['section 2 in the forest conservation act 198...,"['forest conservation act 1980', 'forest conse...",0,[' AIR 1986 AP 225'],"[1436914, 1233720, 1870616, 276791, 1642441, 1...","[(' J Reddy', 'Andhra High Court'), (' J Rao',...",Andhra High Court,Reddy,Anupama Minerals vs Union Of India And Ors. on...,Anupama Minerals,Union Of India And Ors.,1984-12-20,NaN,1
4,107410576,['section 91 in the code of criminal procedure...,['section 91 of the civil procedure code'],0,[],"[704277, 1252855, 1544088, 325366, 487818, 788...",[],Madhya Pradesh High Court,NaN,Nisar Khan vs The State Of Madhya Pradesh on 5...,Nisar Khan,The State Of Madhya Pradesh,2015-10-05,1\n\n IN ...,1


In [40]:
df_non_null_matched.to_csv(citations_data+"possible_air_corpus_with_meta.csv", index=False)